# Selenium ile kariyer.net verilerini çekme

Selenium web sayfaları üzerinden otomasyon, test senaryoları yazmayı sağlayan bir kütüphanedir. Python dışında pek çok programlama dili ile kullanılabilir. Burada Selenium kullanma sebebim kariyer.net web sayfasının dinamik bir arayüzü olması sebebiyle sayfa açıldıktan sonra ekrana yansayacak verileri alabilmektir. Bu sayede daha çok veriyi eksiksiz ve doğru bir şekilde erişebilmiş olacağım.


In [ ]:
#Gerekli fonksiyonları import edelim
import pickle
from dfply import * #dplyr python portu
import time
import random
import pandas as pd
from bs4 import BeautifulSoup # daha ilkel bir html parse kutuphanesi
from selenium import webdriver
with open('urls.pkl', 'rb') as f:
    urls = pickle.load(f)

In [2]:

driver_path = "C:/webdriver/chromedriver.exe"#webdriver.exe konumunu göstermeliyiz. 
driver = webdriver.Chrome(driver_path) 
from selenium.webdriver.common.action_chains import ActionChains
nextpageXpath='//*[@id="__layout"]/div/div[2]/div/div[1]/div[2]/div[2]/div[2]/div[3]/div[27]/div/ul/li[14]/button'
from selenium.webdriver.common.action_chains import ActionChains
def nextpage():
    ActionChains(driver).move_to_element(driver.find_element_by_xpath(nextpageXpath)).perform()
    driver.find_element_by_xpath(nextpageXpath).click()

In [3]:

pozisyon=[]
city=[]
firma=[]
metin=[]
b_sayisi=[]
g_sayisi=[]
sure=[]
link=[]
KriterDF=[]
DT=None

bu variable'lara istenilen bilgiler eklenecektir.Bu sebeble önceden tanınması programatik açıdan gerekmektedir.

In [ ]:
for url in urls:
    driver.get(url)
    time.sleep(0.5)
    driver.execute_script("window.scrollTo(0,700)")
    time.sleep(0.5)
    driver.execute_script("window.scrollTo(0,1000)")
    time.sleep(0.5)
    driver.execute_script("window.scrollTo(0,1300)")
    time.sleep(0.2)
    driver.execute_script("window.scrollTo(0,1600)")
    time.sleep(0.2)
    driver.execute_script("window.scrollTo(0,1800)")
    time.sleep(0.2)
    driver.execute_script("window.scrollTo(0,2300)")
    time.sleep(4)
    driver.execute_script("window.scrollTo(0,1000)")  
    kriterler = {
        "bir": driver.find_elements_by_class_name("col-4"),
        "iki": driver.find_elements_by_class_name("col-12")
    }
    try: 
        pozisyon = pozisyon + [driver.find_element_by_id("jobTitle").text]
    except:
        pozisyon.append(None)
    try:
        city  = city  + [driver.find_element_by_id("jobCity").text]
    except:
        city.append(None)
    try:
        firma = firma + [driver.find_element_by_id("jobCompany").text]
    except:
        firma.append(None)
    try:
        metin = metin + [driver.find_element_by_id("divIlanSablon").text]
    except:
        metin.append(None)
    try:
        b_sayisi=b_sayisi+[driver.find_element_by_xpath("//*[contains(@class, 'basvuru-sayisi')]").find_element_by_css_selector('strong').text]
    except:
        b_sayisi.append(None)
    try:    
        g_sayisi=g_sayisi+[driver.find_element_by_xpath("//*[contains(@class, 'goruntulenme-sayisi')]").find_element_by_css_selector('strong').text]
    except:
        g_sayisi.append(None)
    try:
        sure=sure+[driver.find_element_by_xpath("//*[contains(@class, 'sure')]").find_element_by_css_selector('strong').text]
    except: 
        sure.append(None)
    
    # veriye ait tablo bilgilerini çekme kısmı
    a=[]
    b=[]
    for i in range(len(kriterler['bir'])):
        a.append(kriterler["bir"][i].text)
        b.append(kriterler["iki"][i].text)
    DT=(pd.DataFrame([b],columns=a)>>mutate(urls=url))
    KriterDF.append(DT)
        

Burada selenium her bir url'ye tek tek giderek bu urller içinden ilgili html tag'inden text bilgisini bir değişkene ekler.Bu değişkenin bulunmadığı durumda NaN(missing) veri ekleyerek diğer variableri bulmaya devam eder.

KriterDF değişkeni ise askerlik durumu üniversite durumu her bir ilanın en alt kısmında bulunan kısmı almakla yükümlüdür.

In [18]:
kariyerDF=pd.DataFrame(list(zip(pozisyon,city,firma,metin,b_sayisi,g_sayisi,sure,urls)),
             columns =['pozisyon', 'city','firma','metin','b_sayisi','g_sayisi','sure','urls'])

Burada ilanla alakalı ana bilgiler bir dataframe haline getirilir.

In [25]:
kariyerDF=kariyerDF.drop_duplicates(subset='urls')

Sponsorlu ilanların birden fazla kez bulunması sebebiyle birden fazla kez taranmışlardır.Burada bu duplike ilanlar dataframeden düşürülmektedir.

In [5]:
kriterDF=pd.concat(KriterDF)

C:\Users\fatih\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [22]:
kriterDF=kriterDF.drop_duplicates(subset = ["urls"])

Burada ise kriterDF dictionary veri tipinden dataframe'e çevrilir ve duplike satırlar tekrar temizlenir.

In [29]:
MainDF= pd.merge(kariyerDF, kriterDF, how='inner', left_on='urls', right_on='urls')

Burada MainDF adı altında birleştirilir.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [37]:
MainDF.drop_duplicates(subset='urls')

pozisyon  \
0                     İş Yeri Hekimi (Kayseri)   
1    Bütçe ve Raporlama Uzmanı/ Kıdemli Uzmanı   
2                    Banka Yetkilisi/Yönetmeni   
3     Satış Planlama Uzman Yardımcısı / Uzmanı   
4                           Portföy Yöneticisi   
..                                         ...   
713            Bilgi Sistemleri Denetim Uzmanı   
714                        Java Yazılım Uzmanı   
715                           Planlama Elemanı   
716                                    Analist   
717                            Lojistik Uzmanı   

                                        city  \
0    Adana, Kayseri, Nevşehir, Sivas, Yozgat   
1                 İstanbul(Asya)(Sancaktepe)   
2                   İstanbul(Avr.)(Beşiktaş)   
3                      İstanbul(Avr.)(Şişli)   
4                                      İzmir   
..                                       ...   
713                           İstanbul(Avr.)   
714                 İstanbul(Asya)(Çekmeköy)   
715                İstanbul(Avr.)(Kağıthane)   
716           İstanbul(Avr.), İstanbul(Asya)   
717                           İstanbul(Avr.)   

                                         firma  \
0        VAKIF PAZARLAMA SANAYİ VE TİCARET A.Ş   
1                                Biofarma İlaç   
2                        Destek Faktoring A.Ş.   
3    CK Boğaziçi Elektrik Perakende Satış A.Ş.   
4                            AdresGezgini A.Ş.   
..                                         ...   
713                                    PAYCORE   
714                       ÖZYEĞİN ÜNİVERSİTESİ   
715              Ormoteks Giyim San. Tic. A.Ş.   
716         Vizyoneks Bilgi Teknolojileri A.Ş.   
717                          Çalık Enerji A.Ş.   

                                                 metin   b_sayisi g_sayisi  \
0    GENEL NİTELİKLER VE İŞ TANIMI\nVakıf Pazarlama...       0-50     3062   
1    GENEL NİTELİKLER VE İŞ TANIMI\n\nBiofarma İlaç...       0-50      264   
2    GENEL NİTELİKLER VE İŞ TANIMI\nŞirketimizin ku...       0-50      485   
3    GENEL NİTELİKLER VE İŞ TANIMI\nÜniversitelerin...    100-200      431   
4    GENEL NİTELİKLER VE İŞ TANIMI\nFirma Tanımı:\n...     50-100      510   
..                                                 ...        ...      ...   
713  GENEL NİTELİKLER VE İŞ TANIMI\nŞirketimizin Ay...    200-300     3469   
714  GENEL NİTELİKLER VE İŞ TANIMI\nGENEL NİTELİKLE...    200-300     3707   
715  GENEL NİTELİKLER VE İŞ TANIMI\nÜniversitelerin...  2000-2500    12729   
716  GENEL NİTELİKLER VE İŞ TANIMI\nSigorta Sektörü...  6000-6500    38794   
717  GENEL NİTELİKLER VE İŞ TANIMI\nTercihen üniver...  1500-2000    14691   

     sure                                               urls  \
0       5  https://www.kariyer.net/is-ilani/vakif-pazarla...   
1       0  https://www.kariyer.net/is-ilani/biofarma-ilac...   
2       0  https://www.kariyer.net/is-ilani/destek-faktor...   
3       0  https://www.kariyer.net/is-ilani/ck-bogazici-e...   
4       0  https://www.kariyer.net/is-ilani/adresgezgini-...   
..    ...                                                ...   
713   367  https://www.kariyer.net/is-ilani/paycore-bilgi...   
714   612  https://www.kariyer.net/is-ilani/ozyegin-unive...   
715   879  https://www.kariyer.net/is-ilani/ormoteks-giyi...   
716   997  https://www.kariyer.net/is-ilani/vizyoneks-bil...   
717  1423  https://www.kariyer.net/is-ilani/calik-enerji-...   

     Askerlik Durumu: Company Industry:  ...  Personel Sayısı:  \
0                 NaN               NaN  ...  Belirtilmemiştir   
1             Yapıldı               NaN  ...                 1   
2             Yapıldı               NaN  ...  Belirtilmemiştir   
3             Yapıldı               NaN  ...                 1   
4                 NaN               NaN  ...  Belirtilmemiştir   
..                ...               ...  ...               ...   
713     Muaf, Yapıldı               NaN  ...  Belirtilmemiştir   
714     Yapıldı, Muaf    

In [35]:
MainDF.to_excel('mainDF.xlsx')

R üzerinde analiz yapabilmek için excel formatında veri export edilir.

In [33]:
#with open('MainDF.pkl', 'wb') as f:
#    pickle.dump(MainDF, f) #tablo kaydedildi.